In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
#cargar dataframe
db = sqlite3.connect('30032022-200858.db')
query = "SELECT * FROM Jugador as j INNER JOIN Estadisticas_jugador as e ON e.id_jugador = j.jugador_id"
df = pd.read_sql_query(query, db)
print(df)
db.close()

     jugador_id                     nombre nacionalidad  peso  altura  \
0      3eb22ec9             Bernardo Silva     Portugal  65.0   173.0   
1      3eb22ec9             Bernardo Silva     Portugal  65.0   173.0   
2      3eb22ec9             Bernardo Silva     Portugal  65.0   173.0   
3      3eb22ec9             Bernardo Silva     Portugal  65.0   173.0   
4      3eb22ec9             Bernardo Silva     Portugal  65.0   173.0   
...         ...                        ...          ...   ...     ...   
6512   0d2158d5               Jordan Marié       France  69.0   177.0   
6513   0d2158d5               Jordan Marié       France  69.0   177.0   
6514   0d2158d5               Jordan Marié       France  69.0   177.0   
6515   403f5d20           Olivier Boscagli       France  68.0   181.0   
6516   6b994195  Kévin Théophile-Catherine       France  82.0   183.0   

     fecha_nacimiento  GK  DF  MF  FW  ...  Own_Goals  Own_Goals_percentile  \
0          10/08/1994   0   0   1   1  ...  

In [57]:
print(df.sample(5,axis=0)[['jugador_id','nombre','Goals','Goals_percentile','Assists']].to_latex())

\begin{tabular}{lllrrr}
\toprule
{} & jugador\_id &          nombre &  Goals &  Goals\_percentile &  Assists \\
\midrule
2854 &   64e8ed6d &   Borja Mayoral &   0.17 &              14.0 &     0.11 \\
2113 &   8f696594 &   Memphis Depay &   0.80 &              94.0 &     0.18 \\
3813 &   c2211709 &   Ellyes Skhiri &   0.09 &              59.0 &     0.06 \\
3491 &   e162b013 &  Thomas Meunier &   0.23 &              98.0 &     0.23 \\
857  &   6adbc307 &   Jack Stephens &   0.09 &              88.0 &     0.00 \\
\bottomrule
\end{tabular}



# Data Preparation

## Transformation Pipeline

In [64]:
#drop non numeric columns
df_num = df.drop('jugador_id',axis=1).drop('nombre',axis=1).drop('nacionalidad',axis=1).drop('peso',axis=1).drop('altura',axis=1).drop('fecha_nacimiento',axis=1).drop('GK',axis=1).drop('DF',axis=1).drop('MF',axis=1).drop('FW',axis=1).drop('CB',axis=1).drop('FB',axis=1).drop('DM',axis=1).drop('CM',axis=1).drop('AM',axis=1).drop('WM',axis=1).drop('id_jugador',axis=1).drop('id_campeonato',axis=1).drop('temporada',axis=1)

In [65]:
#Numeric
num_pipeline = Pipeline([
  ('imputer', SimpleImputer(missing_values=np.nan, strategy="median")),
  ('std_scaler', StandardScaler()),
])

In [66]:
#fit numeric only
df_num_tr = num_pipeline.fit_transform(df_num)

In [67]:
df_num_tr_dataframe = pd.DataFrame(df_num_tr, columns=df_num.columns, index=df_num.index)

In [68]:
df_num_tr_dataframe_noPercentile = df_num_tr_dataframe[df_num_tr_dataframe.columns.drop(list(df_num_tr_dataframe.filter(regex='percentile')))]

In [69]:
df_export = df_num_tr_dataframe_noPercentile.copy()

In [70]:
df_export

,Goals,Assists,Non_Penalty_Goals,Penalty_Kicks_Made,Penalty_Kicks_Attempted,Yellow_Cards,Red_Cards,xG,npxG,xA,...,Fouls_Committed,Fouls_Drawn,Offsides,Penalty_Kicks_Won,Penalty_Kicks_Conceded,Own_Goals,Ball_Recoveries,Aerials_won,Aerials_lost,percentage_of_Aerials_Won
0,1.159573,0.739853,1.357538,-0.295222,-0.320102,-1.474186,-0.355154,0.704038,0.588468,0.434422,...,-1.049891,0.673855,-0.681958,1.392924,-0.436844,-0.229797,-0.417708,-1.199071,-0.336345,-2.424588
1,0.435066,1.104933,0.556429,-0.295222,-0.320102,-0.809344,-0.355154,0.379595,0.515153,1.908585,...,-0.652155,-0.440371,-0.472236,0.499803,-0.436844,-0.229797,-0.116680,-0.963890,-0.503854,-1.194403
2,0.713723,1.926364,0.864548,-0.295222,-0.320102,0.150985,-0.355154,1.028482,1.248305,1.785738,...,-0.572608,-0.261071,-0.367375,-0.393319,0.782130,-0.229797,-0.055712,-0.899012,-0.327529,-1.200679
3,-0.289440,1.470014,-0.244679,-0.295222,-0.320102,0.150985,-0.355154,-0.139515,-0.071369,0.802963,...,-0.795340,0.161567,-0.227561,-0.393319,-0.436844,-0.229797,-0.871155,-0.939561,-0.309896,-1.363867
4,0.657991,-0.264119,0.802924,-0.295222,-0.320102,-0.070630,-0.355154,0.639150,0.808413,0.925810,...,-0.477151,-0.248264,0.366651,0.797510,-0.436844,-0.229797,-0.524402,-0.963890,-0.794790,-0.667181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6512,-0.512365,-0.446659,-0.491174,-0.295222,-0.320102,0.298727,-0.355154,-0.593736,-0.584575,-0.671200,...,0.016041,-0.504407,-0.507189,-0.393319,1.086873,-0.229797,0.714005,-1.142303,-0.785974,-1.646308
6513,-0.512365,-0.446659,-0.491174,-0.295222,-0.320102,0.003242,-0.355154,-0.593736,-0.584575,-0.794047,...,-0.270328,-0.709323,-0.681958,-0.393319,-0.436844,-0.229797,1.037896,-0.607063,-0.539119,-0.089746
6514,-0.791021,-0.903010,-0.799293,-0.295222,-0.320102,-0.513858,-0.355154,-0.723514,-0.731206,-0.671200,...,-1.065800,-1.093539,-0.681958,-0.393319,-0.436844,-0.229797,0.565396,-0.582734,-0.697811,0.286841
6515,-0.791021,0.009691,-0.799293,-0.295222,-0.320102,-0.735472,-0.355154,-0.723514,-0.731206,-0.179812,...,-1.097619,1.339829,-0.681958,-0.393319,-0.436844,-0.229797,0.588259,-0.363772,0.139733,-0.503993


In [71]:
df_export.insert(0, "jugador_id", df['jugador_id'], True)
df_export.insert(1, "nombre", df['nombre'], True)
df_export.insert(2, "nacionalidad", df['nacionalidad'], True)
df_export.insert(3, "peso", df['peso'], True)
df_export.insert(4, "altura", df['altura'], True)
df_export.insert(5, "fecha_nacimiento", df['fecha_nacimiento'], True)
df_export.insert(6, "GK", df['GK'], True)
df_export.insert(7, "DF", df['DF'], True)
df_export.insert(8, "MF", df['MF'], True)
df_export.insert(9, "FW", df['FW'], True)
df_export.insert(10, "CB", df['CB'], True)
df_export.insert(11, "FB", df['FB'], True)
df_export.insert(12, "DM", df['DM'], True)
df_export.insert(13, "CM", df['CM'], True)
df_export.insert(14, "AM", df['AM'], True)
df_export.insert(15, "WM", df['WM'], True)
df_export.insert(16, "id_campeonato", df['id_campeonato'], True)
df_export.insert(17, "temporada", df['temporada'], True)
df_export

,jugador_id,nombre,nacionalidad,peso,altura,fecha_nacimiento,GK,DF,MF,FW,...,Fouls_Committed,Fouls_Drawn,Offsides,Penalty_Kicks_Won,Penalty_Kicks_Conceded,Own_Goals,Ball_Recoveries,Aerials_won,Aerials_lost,percentage_of_Aerials_Won
0,3eb22ec9,Bernardo Silva,Portugal,65.0,173.0,10/08/1994,0,0,1,1,...,-1.049891,0.673855,-0.681958,1.392924,-0.436844,-0.229797,-0.417708,-1.199071,-0.336345,-2.424588
1,3eb22ec9,Bernardo Silva,Portugal,65.0,173.0,10/08/1994,0,0,1,1,...,-0.652155,-0.440371,-0.472236,0.499803,-0.436844,-0.229797,-0.116680,-0.963890,-0.503854,-1.194403
2,3eb22ec9,Bernardo Silva,Portugal,65.0,173.0,10/08/1994,0,0,1,1,...,-0.572608,-0.261071,-0.367375,-0.393319,0.782130,-0.229797,-0.055712,-0.899012,-0.327529,-1.200679
3,3eb22ec9,Bernardo Silva,Portugal,65.0,173.0,10/08/1994,0,0,1,1,...,-0.795340,0.161567,-0.227561,-0.393319,-0.436844,-0.229797,-0.871155,-0.939561,-0.309896,-1.363867
4,3eb22ec9,Bernardo Silva,Portugal,65.0,173.0,10/08/1994,0,0,1,1,...,-0.477151,-0.248264,0.366651,0.797510,-0.436844,-0.229797,-0.524402,-0.963890,-0.794790,-0.667181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6512,0d2158d5,Jordan Marié,France,69.0,177.0,29/09/1991,0,0,1,0,...,0.016041,-0.504407,-0.507189,-0.393319,1.086873,-0.229797,0.714005,-1.142303,-0.785974,-1.646308
6513,0d2158d5,Jordan Marié,France,69.0,177.0,29/09/1991,0,0,1,0,...,-0.270328,-0.709323,-0.681958,-0.393319,-0.436844,-0.229797,1.037896,-0.607063,-0.539119,-0.089746
6514,0d2158d5,Jordan Marié,France,69.0,177.0,29/09/1991,0,0,1,0,...,-1.065800,-1.093539,-0.681958,-0.393319,-0.436844,-0.229797,0.565396,-0.582734,-0.697811,0.286841
6515,403f5d20,Olivier Boscagli,France,68.0,181.0,18/11/1997,0,1,0,0,...,-1.097619,1.339829,-0.681958,-0.393319,-0.436844,-0.229797,0.588259,-0.363772,0.139733,-0.503993


In [74]:
print(df_export.iloc[[2854,2113,3813,3491,857]][['jugador_id','nombre','Goals','Assists']].to_latex())

\begin{tabular}{lllrr}
\toprule
{} & jugador\_id &          nombre &     Goals &   Assists \\
\midrule
2854 &   64e8ed6d &   Borja Mayoral &  0.156410 &  0.100962 \\
2113 &   8f696594 &   Memphis Depay &  3.667480 &  0.739853 \\
3813 &   c2211709 &   Ellyes Skhiri & -0.289440 & -0.355389 \\
3491 &   e162b013 &  Thomas Meunier &  0.490798 &  1.196203 \\
857  &   6adbc307 &   Jack Stephens & -0.289440 & -0.903010 \\
\bottomrule
\end{tabular}



In [16]:
df_export.to_json('players_preprocessed_plus_info.json')